# Test Factor Models VaR and ES
This notebook demonstrates how to use:
- A single-factor model Value-at-Risk (VaR) and Expected Shortfall (ES)
- A Fama-French 3-factor model for VaR and ES

It uses historical return data and factor models to estimate portfolio risk.

In [13]:
import data_download as dd
import factor_models as fm
import pandas as pd
import numpy as np

##  Download data and form a portfolio

In [15]:
# Define tickers and shares
tickers = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "JPM",
    "UNH", "KO", "PEP", "BMW.DE", "NESN.SW",
    "NOVN.SW", "ASML.AS"
]

# Assign number of shares per asset
shares = pd.Series(10, index=tickers)

# Download adjusted prices
prices = dd.get_raw_prices(tickers, start="2022-01-01")

# Compute daily value of each position
position_values = prices * shares

# Compute total portfolio value over time
portfolio_value_series = position_values.sum(axis=1)

# Compute asset weights (based on latest available prices)
latest_prices = prices.iloc[-1]
port_val = (shares * latest_prices).sum()
weights = (shares * latest_prices) / port_val

# Compute daily asset returns and portfolio returns
returns = prices.pct_change().dropna()
portfolio_returns = returns @ weights

# Combine historical values into one DataFrame
portfolio_df = position_values.copy()
portfolio_df["PORTFOLIO"] = portfolio_value_series

# Display results
portfolio_df.head()


,AAPL,ASML.AS,BMW.DE,GOOGL,JPM,KO,MSFT,NESN.SW,NOVN.SW,NVDA,PEP,UNH,PORTFOLIO
Date,,,,,,,,,,,,,
2022-01-03,1788.798981,6772.234497,736.722031,1443.015900,1469.958038,537.828331,3256.348267,1150.985565,649.150009,300.682201,1572.986450,4789.945374,24468.655643
2022-01-04,1766.096649,6575.349731,759.792480,1437.124329,1525.683899,546.807327,3200.511169,1153.479538,650.281296,292.386799,1575.260010,4681.421509,24164.194736
2022-01-05,1719.118347,6474.978638,776.328964,1371.194458,1497.790985,551.342125,3077.650452,1122.304382,655.372391,275.556297,1580.625000,4669.882202,23772.144241
2022-01-06,1690.420685,6355.302124,776.167679,1370.920868,1513.703918,548.439865,3053.330688,1119.810410,659.412918,281.286240,1580.988464,4478.772888,23428.556747
2022-01-07,1692.091522,6417.069092,771.166382,1363.650513,1528.702240,547.170067,3054.886475,1108.587341,660.463486,271.992569,1582.989197,4373.395081,23372.163963


## Single-Factor VaR and ES

The Single-Factor model assumes that all asset returns are driven by a single common factor — typically the market index.  
This is also referred to as the **Sharpe Single-Index Model**, and is a special case of the delta-normal method.

Each asset’s return is modeled as:

$$
R_i = \alpha_i + \beta_i R_m + \varepsilon_i
$$

where:
- $R_m$ is the market return
- $\beta_i$ is the sensitivity of asset $i$ to the market
- $\varepsilon_i$ is the idiosyncratic (asset-specific) risk

The total variance of asset $i$ is:

$$
\sigma_i^2 = \beta_i^2 \sigma_m^2 + \sigma_{\varepsilon_i}^2
$$

The portfolio’s covariance matrix is approximated as:

$$
\Sigma = \beta \beta^T \cdot \sigma_m^2 + D
$$

where:
- $\beta$: vector of asset betas
- $\sigma_m^2$: variance of the market return
- $D$: diagonal matrix of idiosyncratic variances

Finally, the portfolio VaR at confidence level $\alpha$ is given by:

$$
\text{VaR}_{\alpha} = z_\alpha \cdot \sigma_p \cdot \text{PortfolioValue}
$$

and the Expected Shortfall (ES) is:

$$
\text{ES}_{\alpha} = \frac{\phi(z_\alpha)}{1 - \alpha} \cdot \sigma_p \cdot \text{PortfolioValue}
$$

where $z_\alpha$ is the quantile of the standard normal distribution and $\phi$ is the standard normal PDF.


In [16]:
# download SPY prices
spy = dd.get_raw_prices(["SPY"], start="2024-01-01")

# compute benchmark returns
benchmark = spy["SPY"].pct_change().dropna()

# align benchmark with asset returns
benchmark = benchmark.reindex(returns.index).ffill()

In [17]:
var_sf, es_sf, Sigma, betas, idio_var = fm.single_factor_var_es(
    returns=returns,
    benchmark=benchmark,
    weights=weights,
    port_val=port_val,
    confidence_level=0.99
)

Sigma.index.name = None
Sigma.columns.name = None

# Print results
print("Single Factor Portfolio VaR (99%):", f"{var_sf:,.2f} USD")
print("Single Factor Portfolio ES  (99%):", f"{es_sf:,.2f} USD\n")

print("Covariance matrix:")
display(Sigma)

print("\n Asset betas relative to SPY:")
display(betas)

print("\n Idiosyncratic variances:")
display(idio_var)

Single Factor Portfolio VaR (99%): 637.26 USD
Single Factor Portfolio ES  (99%): 730.08 USD

Covariance matrix:


,AAPL,ASML.AS,BMW.DE,GOOGL,JPM,KO,MSFT,NESN.SW,NOVN.SW,NVDA,PEP,UNH
AAPL,0.000336,0.000098,0.000025,0.000161,0.000134,1.336527e-05,0.000150,-9.750434e-06,-9.571390e-06,0.000326,1.779874e-05,0.000027
ASML.AS,0.000098,0.000595,0.000014,0.000089,0.000074,7.404068e-06,0.000083,-5.401529e-06,-5.302342e-06,0.000181,9.860115e-06,0.000015
BMW.DE,0.000025,0.000014,0.000301,0.000023,0.000019,1.887578e-06,0.000021,-1.377055e-06,-1.351768e-06,0.000046,2.513718e-06,0.000004
GOOGL,0.000161,0.000089,0.000023,0.000429,0.000122,1.214931e-05,0.000137,-8.863351e-06,-8.700596e-06,0.000296,1.617943e-05,0.000025
JPM,0.000134,0.000074,0.000019,0.000122,0.000259,1.016382e-05,0.000114,-7.414862e-06,-7.278705e-06,0.000248,1.353532e-05,0.000021
KO,0.000013,0.000007,0.000002,0.000012,0.000010,1.025081e-04,0.000011,-7.377153e-07,-7.241688e-07,0.000025,1.346648e-06,0.000002
MSFT,0.000150,0.000083,0.000021,0.000137,0.000114,1.138538e-05,0.000308,-8.306035e-06,-8.153514e-06,0.000278,1.516209e-05,0.000023
NESN.SW,-0.000010,-0.000005,-0.000001,-0.000009,-0.000007,-7.377153e-07,-0.000008,1.163617e-04,5.283067e-07,-0.000018,-9.824271e-07,-0.000001
NOVN.SW,-0.000010,-0.000005,-0.000001,-0.000009,-0.000007,-7.241688e-07,-0.000008,5.283067e-07,1.268573e-04,-0.000018,-9.643871e-07,-0.000001
NVDA,0.000326,0.000181,0.000046,0.000296,0.000248,2.466702e-05,0.000278,-1.799546e-05,-1.766501e-05,0.001254,3.284946e-05,0.000050



 Asset betas relative to SPY:


AAPL       1.172892
ASML.AS    0.649756
BMW.DE     0.165648
GOOGL      1.066183
JPM        0.891943
KO         0.088741
MSFT       0.999143
NESN.SW   -0.064739
NOVN.SW   -0.063551
NVDA       2.164696
PEP        0.118177
UNH        0.179380
dtype: float64


 Idiosyncratic variances:


AAPL       0.000160
ASML.AS    0.000541
BMW.DE     0.000297
GOOGL      0.000283
JPM        0.000156
KO         0.000101
MSFT       0.000180
NESN.SW    0.000116
NOVN.SW    0.000126
NVDA       0.000652
PEP        0.000124
UNH        0.000301
dtype: float64

## Fama-French 3-Factor VaR and ES

The Fama–French 3-factor model extends the CAPM by modeling asset returns using three sources of systematic risk:

- **Mkt_RF**: excess return of the market over the risk-free rate  
- **SMB** ("Small Minus Big"): return of small-cap stocks minus large-cap  
- **HML** ("High Minus Low"): return of high book-to-market stocks minus low  

Each asset’s excess return is regressed on these three factors to estimate its **exposures (betas)**. The model assumes:

$$
R_i - R_f = \alpha_i + \beta_{i1} \cdot \text{Mkt}_{RF} + \beta_{i2} \cdot \text{SMB} + \beta_{i3} \cdot \text{HML} + \varepsilon_i
$$

The estimated covariance matrix of asset returns is reconstructed as:

$$
\Sigma = B \cdot \Sigma_f \cdot B^T + D
$$

Where:
- $B$: matrix of asset betas  
- $\Sigma_f$: covariance matrix of factor returns  
- $D$: diagonal matrix of idiosyncratic variances  

The next code block performs the following steps:

1. Downloads the FF3 daily factor data from Ken French's database.
2. Computes **excess returns** for each asset over the risk-free rate.
3. Regresses each asset’s excess return on the three factors to obtain:
   - **Beta coefficients** (exposures to Mkt_RF, SMB, HML)
   - **Idiosyncratic variance** (residual risk)
4. Builds the **full covariance matrix** of asset returns using the factor structure.
5. Computes **portfolio volatility** using the covariance matrix and current weights.
6. Calculates:
   - **Value-at-Risk (VaR)** at the given confidence level
   - **Conditional VaR (CVaR)**, i.e. expected loss beyond the VaR threshold

The printed outputs show all key components estimated during this process.

In [18]:
# Download FF3 daily factors and restrict to portfolio date range
ff_factors = fm.load_ff3_factors(start=returns.index[0], end=returns.index[-1])

# Check first few rows
ff_factors.head()

,Mkt_RF,SMB,HML,RF
2022-01-04,-0.0029,-0.0083,0.0364,0.0
2022-01-05,-0.0228,-0.0145,0.0259,0.0
2022-01-06,0.0000,0.0020,0.0175,0.0
2022-01-07,-0.0048,-0.0132,0.0202,0.0
2022-01-10,-0.0015,-0.0023,-0.0028,0.0


In [19]:
var_ff, es_ff, ff_betas, ff_resid, ff_cov = fm.ff3_var_es(
    prices=prices, shares=shares, alpha=0.99
)

print(f"FF3 VaR  (99%): {var_ff:,.2f} USD")
print(f"FF3 ES (99%): {es_ff:,.2f} USD\n")

print("FF3 Betas:")
display(ff_betas)

print("\n Idiosyncratic variances:")
display(ff_resid)

print("\n FF3 Covariance Matrix (Σ):")
display(pd.DataFrame(ff_cov, index=ff_betas.index, columns=ff_betas.index))

FF3 VaR  (99%): 703.96 USD
FF3 ES (99%): 806.51 USD

FF3 Betas:


,Mkt_RF,SMB,HML
AAPL,1.073133,-0.251177,-0.268468
ASML.AS,0.982026,-0.149765,-0.189005
BMW.DE,0.471534,0.106558,0.289300
GOOGL,1.167405,-0.299357,-0.371906
JPM,1.071320,-0.110136,0.721975
KO,0.445422,-0.269201,0.173037
MSFT,1.089975,-0.460126,-0.398944
NESN.SW,0.161012,-0.118375,0.090844
NOVN.SW,0.196541,-0.122009,0.186663
NVDA,1.907581,-0.495636,-0.869908



 Idiosyncratic variances:


AAPL       0.000181
ASML.AS    0.000469
BMW.DE     0.000276
GOOGL      0.000234
JPM        0.000145
KO         0.000085
MSFT       0.000136
NESN.SW    0.000114
NOVN.SW    0.000122
NVDA       0.000664
PEP        0.000107
UNH        0.000282
dtype: float64


 FF3 Covariance Matrix (Σ):


,AAPL,ASML.AS,BMW.DE,GOOGL,JPM,KO,MSFT,NESN.SW,NOVN.SW,NVDA,PEP,UNH
AAPL,0.000336,0.000140,0.000046,0.000174,0.000094,0.000043,0.000162,0.000013,0.000012,0.000301,0.000046,0.000051
ASML.AS,0.000140,0.000595,0.000044,0.000156,0.000089,0.000039,0.000145,0.000012,0.000011,0.000270,0.000041,0.000047
BMW.DE,0.000046,0.000044,0.000301,0.000050,0.000052,0.000018,0.000044,0.000006,0.000008,0.000080,0.000016,0.000021
GOOGL,0.000174,0.000156,0.000050,0.000429,0.000101,0.000047,0.000182,0.000015,0.000012,0.000338,0.000051,0.000056
JPM,0.000094,0.000089,0.000052,0.000101,0.000259,0.000041,0.000091,0.000015,0.000020,0.000160,0.000038,0.000049
KO,0.000043,0.000039,0.000018,0.000047,0.000041,0.000103,0.000045,0.000007,0.000008,0.000078,0.000018,0.000021
MSFT,0.000162,0.000145,0.000044,0.000182,0.000091,0.000045,0.000308,0.000014,0.000012,0.000317,0.000048,0.000052
NESN.SW,0.000013,0.000012,0.000006,0.000015,0.000015,0.000007,0.000014,0.000116,0.000003,0.000023,0.000006,0.000007
NOVN.SW,0.000012,0.000011,0.000008,0.000012,0.000020,0.000008,0.000012,0.000003,0.000127,0.000018,0.000007,0.000009
NVDA,0.000301,0.000270,0.000080,0.000338,0.000160,0.000078,0.000317,0.000023,0.000018,0.001254,0.000084,0.000092
